<a href="https://colab.research.google.com/github/LC1332/prompt-ng-andrew/blob/main/content/5.%20%E6%8E%A8%E6%96%AD%20Inferring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 吴恩达老师的Prompt Engineering课程学习

本项目由李鲁鲁老师在DataWhale翻译的中文版本的吴恩达老师与 OpenAI 合作推出的 《ChatGPT Prompt Engineering for Developers》基础上，进行了一些自己的实践思考。

项目链接 [https://github.com/LC1332/prompt-ng-andrew](https://github.com/LC1332/prompt-ng-andrew)

原来DataWhale项目的链接是 [https://github.com/datawhalechina/prompt-engineering-for-developers](https://github.com/datawhalechina/prompt-engineering-for-developers)

每一课都在colab上进行了改造，并将链接放在了readme，可以直接用colab进行学习。

In [ ]:
! pip install openai
! pip install python-dotenv

# 推断

在这节课中，你将从产品评论和新闻文章中推断情感和主题。

这些任务可以看作是模型接收文本作为输入并执行某种分析的过程。这可能涉及提取标签、提取实体、理解文本情感等等。如果你想要从一段文本中提取正面或负面情感，在传统的机器学习工作流程中，需要收集标签数据集、训练模型、确定如何在云端部署模型并进行推断。这样做可能效果还不错，但是这个过程需要很多工作。而且对于每个任务，如情感分析、提取实体等等，都需要训练和部署单独的模型。

大型语言模型的一个非常好的特点是，对于许多这样的任务，你只需要编写一个prompt即可开始产生结果，而不需要进行大量的工作。这极大地加快了应用程序开发的速度。你还可以只使用一个模型和一个 API 来执行许多不同的任务，而不需要弄清楚如何训练和部署许多不同的模型。


## 启动

In [2]:
import openai
import os

# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY2")
openai.api_key = 'sk-' # 在这里输入你的OpenAI API token

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

## 商品评论文本

这是一盏台灯的评论。

In [4]:
lamp_review = """
Needed a nice lamp for my bedroom, and this one had \
additional storage and not too high of a price point. \
Got it fast.  The string to our lamp broke during the \
transit and the company happily sent over a new one. \
Came within a few days as well. It was easy to put \
together.  I had a missing part, so I contacted their \
support and they very quickly got me the missing piece! \
Lumina seems to me to be a great company that cares \
about their customers and products!!
"""

In [5]:
# 中文
lamp_review_zh = """
我需要一盏漂亮的卧室灯，这款灯具有额外的储物功能，价格也不算太高。\
我很快就收到了它。在运输过程中，我们的灯绳断了，但是公司很乐意寄送了一个新的。\
几天后就收到了。这款灯很容易组装。我发现少了一个零件，于是联系了他们的客服，他们很快就给我寄来了缺失的零件！\
在我看来，Lumina 是一家非常关心顾客和产品的优秀公司！
"""

## 情感（正向/负向）

现在让我们来编写一个prompt来分类这个评论的情感。如果我想让系统告诉我这个评论的情感是什么，只需要编写 “以下产品评论的情感是什么” 这个prompt，加上通常的分隔符和评论文本等等。

然后让我们运行一下。结果显示这个产品评论的情感是积极的，这似乎是非常正确的。虽然这盏台灯不完美，但这个客户似乎非常满意。这似乎是一家关心客户和产品的伟大公司，可以认为积极的情感似乎是正确的答案。

In [6]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

The sentiment of the product review is positive.


In [7]:
# 中文
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

情感是积极的/正面的。


如果你想要给出更简洁的答案，以便更容易进行后处理，可以使用上面的prompt并添加另一个指令，以一个单词 “正面” 或 “负面” 的形式给出答案。这样就只会打印出 “正面” 这个单词，这使得文本更容易接受这个输出并进行处理。

In [8]:
prompt = f"""
What is the sentiment of the following product review, 
which is delimited with triple backticks?

Give your answer as a single word, either "positive" \
or "negative".

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

positive


In [10]:
prompt = f"""
以下用三个反引号分隔的产品评论的情感是什么？

用一个单词回答："正面"或"负面"。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

正面


## 识别情感类型

让我们看看另一个prompt，仍然使用台灯评论。这次我要让它识别出以下评论作者所表达的情感列表，不超过五个。

In [11]:
prompt = f"""
Identify a list of emotions that the writer of the \
following review is expressing. Include no more than \
five items in the list. Format your answer as a list of \
lower-case words separated by commas.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

satisfied, grateful, impressed, content, pleased


In [12]:
# 中文
prompt = f"""
识别以下评论的作者表达的情感。包含不超过五个项目。将答案格式化为以逗号分隔的单词列表。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

满意,感激,信任,赞扬,愉快


大型语言模型非常擅长从一段文本中提取特定的东西。在上面的例子中，评论正在表达情感，这可能有助于了解客户如何看待特定的产品。

李鲁鲁： 这里我有点好奇拿修仙小说里面的描述，会怎么样，让我们来试一下

这个片段是老魔第一次看到南宫婉

In [13]:
fiction_text = """
一个狭长的地下山洞内，悄然无声的走着一行白衣男女，看人数约有十五六人左右，竟是所有禁地内活下来的掩月宗弟子都在了此地，而走在最前面的，就是曾远远见过韩立一面的那位精灵一样的少女！

白衣少女虽然面似童真清纯，此时却神情肃然，全身上下笼罩在了一层奇异的银辉之下，让人显得越发的神秘诡异！

最让人惊讶的是，跟在少女身后的白衣男女全都一副兢兢战战的模样，竟连一个窃窃私语小声嘀咕的都没有，望向少女背影的眼神更是充满了敬畏之色！

而曾在沙地出现过的刁蛮女子与她的修炼道侣也身在其中，只是脸上的骄横之色已无影无踪，和旁人一样的大气也不敢喘一下，显得格外的乖巧老实！
"""

prompt = f"""
识别以下评论的作者表达的情感。包含不超过五个项目。将答案格式化为以逗号分隔的单词列表。

评论文本: ```{fiction_text}```
"""
response = get_completion(prompt)
print(response)

神秘, 敬畏, 惊讶, 乖巧, 肃然


## 识别愤怒

对于很多企业来说，了解某个顾客是否非常生气很重要。所以你可能有一个类似这样的分类问题：以下评论的作者是否表达了愤怒情绪？因为如果有人真的很生气，那么可能值得额外关注，让客户支持或客户成功团队联系客户以了解情况，并为客户解决问题。

In [14]:
prompt = f"""
Is the writer of the following review expressing anger?\
The review is delimited with triple backticks. \
Give your answer as either yes or no.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

No


In [15]:
# 中文
prompt = f"""
以下评论的作者是否表达了愤怒？评论用三个反引号分隔。给出是或否的答案。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

否


上面这个例子中，客户并没有生气。注意，如果使用常规的监督学习，如果想要建立所有这些分类器，不可能在几分钟内就做到这一点。我们鼓励大家尝试更改一些这样的prompt，也许询问客户是否表达了喜悦，或者询问是否有任何遗漏的部分，并看看是否可以让prompt对这个灯具评论做出不同的推论。

## 从客户评论中提取产品和公司名称

接下来，让我们从客户评论中提取更丰富的信息。信息提取是自然语言处理（NLP）的一部分，与从文本中提取你想要知道的某些事物相关。因此，在这个prompt中，我要求它识别以下内容：购买物品和制造物品的公司名称。

同样，如果你试图总结在线购物电子商务网站的许多评论，对于这些评论来说，弄清楚是什么物品，谁制造了该物品，弄清楚积极和消极的情感，以跟踪特定物品或特定制造商的积极或消极情感趋势，可能会很有用。

在下面这个示例中，我们要求它将响应格式化为一个 JSON 对象，其中物品和品牌是键。

In [16]:
prompt = f"""
Identify the following items from the review text: 
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Item" and "Brand" as the keys. 
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
  
Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

{
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


In [17]:
# 中文
prompt = f"""
从评论文本中识别以下项目：
- 评论者购买的物品
- 制造该物品的公司

评论文本用三个反引号分隔。将你的响应格式化为以 “物品” 和 “品牌” 为键的 JSON 对象。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
  
评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

{
  "物品": "卧室灯",
  "品牌": "Lumina"
}


如上所示，它会说这个物品是一个卧室灯，品牌是 Luminar，你可以轻松地将其加载到 Python 字典中，然后对此输出进行其他处理。

## 一次完成多项任务

提取上面所有这些信息使用了 3 或 4 个prompt，但实际上可以编写单个prompt来同时提取所有这些信息。

In [18]:
prompt = f"""
Identify the following items from the review text: 
- Sentiment (positive or negative)
- Is the reviewer expressing anger? (true or false)
- Item purchased by reviewer
- Company that made the item

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Sentiment", "Anger", "Item" and "Brand" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: ```{lamp_review}```
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "positive",
  "Anger": false,
  "Item": "lamp with additional storage",
  "Brand": "Lumina"
}


In [19]:
# 中文
prompt = f"""
从评论文本中识别以下项目：
- 情绪（正面或负面）
- 审稿人是否表达了愤怒？（是或否）
- 评论者购买的物品
- 制造该物品的公司

评论用三个反引号分隔。将您的响应格式化为 JSON 对象，以 “Sentiment”、“Anger”、“Item” 和 “Brand” 作为键。
如果信息不存在，请使用 “未知” 作为值。
让你的回应尽可能简短。
将 Anger 值格式化为布尔值。

评论文本: ```{lamp_review_zh}```
"""
response = get_completion(prompt)
print(response)

{
  "Sentiment": "正面",
  "Anger": false,
  "Item": "卧室灯",
  "Brand": "Lumina"
}


这个例子中，我们告诉它将愤怒值格式化为布尔值，然后输出一个 JSON。大家可以自己尝试不同的变化，或者甚至尝试完全不同的评论，看看是否仍然可以准确地提取这些内容。

## 推断主题

大型语言模型的一个很酷的应用是推断主题。给定一段长文本，这段文本是关于什么的？有什么话题？

In [20]:
story = """
In a recent survey conducted by the government, 
public sector employees were asked to rate their level 
of satisfaction with the department they work at. 
The results revealed that NASA was the most popular 
department with a satisfaction rating of 95%.

One NASA employee, John Smith, commented on the findings, 
stating, "I'm not surprised that NASA came out on top. 
It's a great place to work with amazing people and 
incredible opportunities. I'm proud to be a part of 
such an innovative organization."

The results were also welcomed by NASA's management team, 
with Director Tom Johnson stating, "We are thrilled to 
hear that our employees are satisfied with their work at NASA. 
We have a talented and dedicated team who work tirelessly 
to achieve our goals, and it's fantastic to see that their 
hard work is paying off."

The survey also revealed that the 
Social Security Administration had the lowest satisfaction 
rating, with only 45% of employees indicating they were 
satisfied with their job. The government has pledged to 
address the concerns raised by employees in the survey and 
work towards improving job satisfaction across all departments.
"""

In [21]:
# 中文
story_zh = """
在政府最近进行的一项调查中，要求公共部门的员工对他们所在部门的满意度进行评分。
调查结果显示，NASA 是最受欢迎的部门，满意度为 95％。

一位 NASA 员工 John Smith 对这一发现发表了评论，他表示：
“我对 NASA 排名第一并不感到惊讶。这是一个与了不起的人们和令人难以置信的机会共事的好地方。我为成为这样一个创新组织的一员感到自豪。”

NASA 的管理团队也对这一结果表示欢迎，主管 Tom Johnson 表示：
“我们很高兴听到我们的员工对 NASA 的工作感到满意。
我们拥有一支才华横溢、忠诚敬业的团队，他们为实现我们的目标不懈努力，看到他们的辛勤工作得到回报是太棒了。”

调查还显示，社会保障管理局的满意度最低，只有 45％的员工表示他们对工作满意。
政府承诺解决调查中员工提出的问题，并努力提高所有部门的工作满意度。
"""

## 推断5个主题

上面是一篇虚构的关于政府工作人员对他们工作机构感受的报纸文章。我们可以让它确定五个正在讨论的主题，用一两个字描述每个主题，并将输出格式化为逗号分隔的列表。

In [22]:
prompt = f"""
Determine five topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: ```{story}```
"""
response = get_completion(prompt)
print(response)

government survey, public sector employees, job satisfaction, NASA, Social Security Administration


In [23]:
response.split(sep=',')

['government survey',
 ' public sector employees',
 ' job satisfaction',
 ' NASA',
 ' Social Security Administration']

In [24]:
# 中文
prompt = f"""
确定以下给定文本中讨论的五个主题。

每个主题用1-2个单词概括。

输出时用逗号分割每个主题。

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

公共部门满意度调查, NASA, 社会保障管理局, 员工满意度, 政府承诺


李鲁鲁：我们继续拿凡人修仙传来玩一下



In [25]:
long_fiction = """
修成之后，这门神通在对敌时，能借助飞剑的剑光，另行幻化出一道和飞剑一模一样的剑影来，可迷幻敌人的视线并跟随本体一同攻击敌人。虽然剑影初成时，只有本体的威力十分之一，但随着剑诀层次的提升，其威力还可增加，到了第九层时就可有了三分之一的威力了。

    而且剑影在修炼时，可幻化的并不止一道，从第七层开始时每升一层就可多炼化出一道剑影。如此一来，青元剑诀炼至了极致，就可同时有三道和飞剑外观一样，但威力只有三分之一的剑影。

    这样看来，这“剑光分影术”的神通似乎不赖，还可一修的样子。

    但韩立已经知道，黄枫谷这么多筑基期的修士，并没有谁真的将其深炼下去，这其中的猫腻肯定小不了！因此他大为后悔，为何当初没有打听清楚其中的缘由，只以为肯定不会修炼这青元剑诀，所以就大大咧咧的马虎过去了。

    到如今，韩立虽然明知此法决大有问题，可还不得不硬着头皮去修炼一下，只希望此法决可别有什么类似走火入魔的后患才行。

    不过他也转念想过，其他人虽然没有深加修炼，可也有两三层在身的样子。这样看来，稍微修炼一下也没什么大问题才对。

    抱着这种自我安慰的想法，韩立无奈的按照青元剑诀的法门，吸纳起了体内快要发作的药力。

    此功法只简单的运行了一个循环，韩立就感到轰得一下，这种吸纳了药力让法力暴涨的感觉，让他舒畅的差点叫出声来！

    沉浸在这种美妙滋味中的韩立，不由得让法决一遍遍的循环下去，而神智却渐渐的神游天外了。

    不知打坐了多长时间，当韩立将体内的最后一丝药力吸纳干净的时候，终于从种美妙的体验中苏醒了过来。

    清醒过来的韩立，稍微呆滞了一下，但随后二话不说的站起身来。然后眯起眼睛歪头想了一下，就突然抬起手臂在身前比划了一下，顿时从手指上窜出了尺许长的一截青濛濛剑芒，寒气逼人，锋利无比的样子。

    见此寒光，韩立非但没高兴，反而苦笑了起来！然后手臂突然一甩，青光竟然猛然暴涨，一下变成了一丈多长，几乎就要插入对面的石墙内。

     “这下糟了，没想到残余的药力这么强，竟然一下炼成了第四层的剑诀，不知道会不会有大碍啊！”韩立脸上阴晴不定的嘀咕道。

    “不管了，顶多以后不再修炼此剑决就是了！”韩立喃喃的说道，接着把手臂一放，青色剑芒就消失了。

    不过，好奇心大起的韩立，还是捡起那本《青元剑诀》，翻看了有关护体剑盾的法决并，并默记了几遍。

    接着，韩立低头沉思了一下后，闭起了双目，紧接着再猛一睁眼，身上已出现了一个奇特的护盾。

    此护盾通体青色，和普通的防御罩差不多大小，但表面不再是通常的平滑形态，而表现出刺猬一样的芒刺状，看上去隐隐有种煞气透露出来。

    “这就是护体剑盾？”韩立仔细观察着身上的刺盾，有些惊讶。

    “剑诀上说，此盾可以自行释放剑芒，反击敌人。可惜现在无法测试一下！”韩立颇为遗憾的想道。
"""

prompt = f"""
确定以下给定文本中讨论的五个主题。

每个主题用1-2个单词概括。

输出时用逗号分割每个主题。

给定文本: ```{long_fiction}```
"""
response = get_completion(prompt)
print(response)

剑光分影术, 青元剑诀, 药力暴涨, 护体剑盾, 反击敌人


李鲁鲁：其实还挺好玩儿的。。。

## 为特定主题制作新闻提醒

假设我们有一个新闻网站或类似的东西，这是我们感兴趣的主题：NASA、地方政府、工程、员工满意度、联邦政府等。假设我们想弄清楚，针对一篇新闻文章，其中涵盖了哪些主题。可以使用这样的prompt：确定以下主题列表中的每个项目是否是以下文本中的主题。以 0 或 1 的形式给出答案列表。

In [26]:
topic_list = [
    "nasa", "local government", "engineering", 
    "employee satisfaction", "federal government"
]

In [27]:
prompt = f"""
Determine whether each item in the following list of \
topics is a topic in the text below, which
is delimited with triple backticks.

Give your answer as list with 0 or 1 for each topic.\

List of topics: {", ".join(topic_list)}

Text sample: ```{story}```
"""
response = get_completion(prompt)
print(response)

nasa: 1
local government: 0
engineering: 0
employee satisfaction: 1
federal government: 1


In [28]:
topic_dict = {i.split(': ')[0]: int(i.split(': ')[1]) for i in response.split(sep='\n')}
if topic_dict['nasa'] == 1:
    print("ALERT: New NASA story!")

ALERT: New NASA story!


In [29]:
# 中文
prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，

以列表的形式给出答案，每个主题用 0 或 1。

主题列表：美国航空航天局、地方政府、工程、员工满意度、联邦政府

给定文本: ```{story_zh}```
"""
response = get_completion(prompt)
print(response)

美国航空航天局：1
地方政府：1
工程：0
员工满意度：1
联邦政府：1


所以，这个故事是关于 NASA 的。它不是关于地方政府的，不是关于工程的。它是关于员工满意度的，它是关于联邦政府的。这在机器学习中有时被称为 Zero-Shot 学习算法，因为我们没有给它任何标记的训练数据。仅凭prompt，它就能确定哪些主题在新闻文章中涵盖了。

如果我们想生成一个新闻提醒，也可以使用这个处理新闻的过程。假设我非常喜欢 NASA 所做的工作，就可以构建一个这样的系统，每当 NASA 新闻出现时，输出提醒。

In [30]:
topic_dict = {i.split('：')[0]: int(i.split('：')[1]) for i in response.split(sep='\n')}
if topic_dict['美国航空航天局'] == 1:
    print("提醒: 关于美国航空航天局的新消息")

提醒: 关于美国航空航天局的新消息


这就是关于推断的全部内容了，仅用几分钟时间，我们就可以构建多个用于对文本进行推理的系统，而以前则需要熟练的机器学习开发人员数天甚至数周的时间。这非常令人兴奋，无论是对于熟练的机器学习开发人员还是对于新手来说，都可以使用prompt来非常快速地构建和开始相当复杂的自然语言处理任务。

李鲁鲁：可以试一下凡人那个文本

In [31]:
# 中文
prompt = f"""
判断主题列表中的每一项是否是给定文本中的一个话题，

以列表的形式给出答案，每个主题用 0 或 1。

主题列表：黄枫谷、剑法、拳法、少林派、修仙、修炼、主动技能、防御技能、功法修炼、内功修炼、炼器、炼丹、种植草药

给定文本: ```{long_fiction}```
"""
response = get_completion(prompt)
print(response)

黄枫谷：1
剑法：1
拳法：0
少林派：0
修仙：1
修炼：1
主动技能：1
防御技能：1
功法修炼：1
内功修炼：0
炼器：0
炼丹：0
种植草药：0


## 尝试你的实验！

李鲁鲁：这一课其实非常有趣，特别后面那个一次性多分类的真的是学习了。